Get required libraries

In [1]:
pip install transformers datasets torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 19.3 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Load data from Huggingface

In [4]:
from datasets import load_dataset
import os

current_wd = os.getcwd()
dataset_test = load_dataset("LouisJ444/dutchSQuAD", field="data")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/LouisJ444___json/LouisJ444--dutchSQuAD-152e92ff4d4f28fc/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset_test

DatasetDict({
    train: Dataset({
        features: ['context', 'id', 'title', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['context', 'id', 'title', 'question', 'answers'],
        num_rows: 9380
    })
    test: Dataset({
        features: ['context', 'id', 'title', 'question', 'answers'],
        num_rows: 1183
    })
})

Load in BERTje as model (Autotokenizer in this case)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")

Setup Preprocess function that reads in the begin and end position of answers inside a paragraph (Source: [Huggingface](https://huggingface.co/learn/nlp-course/chapter7/7?fw=tf))

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Set datacollator

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()


Preprocess the datasets

In [ ]:
tokenized_val = dataset_test["validation"].map(preprocess_function, batched=True, remove_columns=dataset_test["validation"].column_names)
tokenized_train = dataset_test["train"].map(preprocess_function, batched=True, remove_columns=dataset_test["train"].column_names)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Download the BERTje model

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("GroNLP/bert-base-dutch-cased")



Some weights of the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are

Set CUDA if possible, else use CPU

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Set trainer and make sure it uses GPU

NOTE:
Dev set is used as validation set!
This was a mistake and a separate set of the Training should have been taken like with the other models.
Due to time constraints, this was not fixed.

In [ ]:
training_args = TrainingArguments(
    output_dir="LouisJ444/Dutch_nlSQuAD_BERTje",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.args.device

device(type='cuda', index=0)

Start training the model

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.374900,1.297912
2,1.066100,1.287475


TrainOutput(global_step=7300, training_loss=1.3254484662617723, metrics={'train_runtime': 12459.6946, 'train_samples_per_second': 14.061, 'train_steps_per_second': 0.586, 'total_flos': 3.4334001889975296e+16, 'train_loss': 1.3254484662617723, 'epoch': 2.0})

Push to huggingface if possible, else download it

In [ ]:
trainer.push_to_hub("LouisJ444/Dutch_nlSQuAD")

Loss Results:

1 	1.374900 	1.297912

2 	1.066100 	1.287475

Use Dev set for testing

In [5]:
from transformers import pipeline
from datasets import load_metric
# qa = pipeline("question-answering", model=f"{current_wd}/LouisJ444/Dutch_nlSQuAD_BERTje/checkpoint-7000", device=0)
qa = pipeline("question-answering", model="LouisJ444/bertje_dutchsquad", device=0)
squad_metric = load_metric("squad")
test_dataset = dataset_test["validation"]

preds = []
refs = []
for item in test_dataset:
  question = item["question"]
  context = item["context"]
  id = item["id"]
  pred_raw = qa(question=question, context=context)
  pred = {"prediction_text": pred_raw["answer"], "id": id}
  ref = {"answers": item["answers"], "id": id}
  refs.append(ref)
  preds.append(pred)

squad_metric.compute(references=refs, predictions=preds)

  

<ipython-input-5-caf0ec26a686>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'exact_match': 68.8272921108742, 'f1': 80.61343927278145}

'exact_match': 68.8272921108742, 'f1': 80.61343927278145